In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [2]:
conf = (SparkConf().setAppName('PartitionIdFunc').setMaster('yarn') \
    .set('spark.sql.adaptive.enabled', 'false'))

spark = SparkSession.builder.config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/17 20:14:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/17 20:14:30 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
24/12/17 20:14:30 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
sc = spark.sparkContext
print("app_id".ljust(40), sc.applicationId)

app_id                                   application_1727681258360_0049


In [4]:
from pyspark.sql.functions import spark_partition_id, count, collect_list
# create DataFrame with 40 values with 4 partitions  
data = [ [k] for k in range(0, 40) ] # transform to 2-D list
df = spark.sparkContext.parallelize(data, 4).toDF(["val"])
df = df.withColumn("row_partition_num", spark_partition_id())
df.groupBy("row_partition_num").agg(
    count("*").alias("count"),
    collect_list('val').alias('val_list')\
   ).orderBy("row_partition_num").show(truncate=False)

+-----------------+-----+----------------------------------------+
|row_partition_num|count|val_list                                |
+-----------------+-----+----------------------------------------+
|0                |10   |[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]          |
|1                |10   |[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]|
|2                |10   |[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]|
|3                |10   |[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]|
+-----------------+-----+----------------------------------------+



In [5]:
# repartition to 5 partitions
df = df.repartition(5).withColumn("row_partition_num", spark_partition_id())
df.groupBy("row_partition_num").agg(
     count("*").alias("count"),
     collect_list('val').alias('val_list')\
    ).orderBy("row_partition_num").show(truncate=False)

+-----------------+-----+------------------------------+
|row_partition_num|count|val_list                      |
+-----------------+-----+------------------------------+
|0                |8    |[7, 2, 11, 16, 22, 21, 35, 32]|
|1                |8    |[14, 18, 20, 29, 39, 34, 0, 9]|
|2                |8    |[13, 12, 28, 24, 37, 36, 5, 4]|
|3                |8    |[15, 17, 25, 27, 38, 31, 6, 1]|
|4                |8    |[19, 10, 26, 23, 33, 30, 3, 8]|
+-----------------+-----+------------------------------+



In [6]:
spark.stop()